# ALI Performance Tests on Blake

In [1]:
# https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import HTML
button = HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show/hide code blocks"></form>''')

In [2]:
from utils import * 
hide_code_button()

In [10]:
import datetime as dt
import glob
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import os
import json

import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

# Import scripts
from json2timeline import json2dataframe
from models import findChangePts
from basicstats import regimeTimeseries

In [11]:
# Enable offline plot export
init_notebook_mode(connected=True)

## Specifications

In [12]:
# Cases to plot:
cases = ('ant-2-20km_ml_ls','ant-2-20km_mu_ls','ant-2-20km_mu_dls')
nproc = 384
# names = ('Total Time',
#          'Total Fill Time',
#          'NOX Total Preconditioner Construction',
#          'NOX Total Linear Solve')
# timers = ('Albany Total Time:',
#           'Albany: Total Fill Time:',
#           'NOX Total Preconditioner Construction:',
#           'NOX Total Linear Solve:')
metadata = ('Albany cxx compiler',
            'Albany git commit id',
            'Trilinos git commit id')

cases = ('ant-2-20km_ml_ls',
         'ant-2-20km_mu_ls',
         'ant-2-20km_mu_dls',
         'green-1-7km_fea_1ws',
         'green-1-7km_ml_ls_1ws',
         'green-1-7km_mu_ls_1ws',
         'green-1-7km_mu_dls_1ws',
         'green-1-7km_fea_mem',
         'green-1-7km_ml_ls_mem',
         'green-1-7km_mu_ls_mem',
         'green-1-7km_mu_dls_mem')
names =  ('Total Time',
          'Setup Time',
          'Total Fill Time',
          'Residual Fill',
          'Residual Fill Evaluate',
          'Residual Fill Export',
          'Jacobian Fill',
          'Jacobian Fill Evaluate',
          'Jacobian Fill Export',
          'NOX Total Preconditioner Construction',
          'NOX Total Linear Solve')
timers = ('Albany Total Time:',
          'Albany: Setup Time:',
          'Albany: Total Fill Time:',
          'Albany Fill: Residual:',
          'Albany Residual Fill: Evaluate:',
          'Albany Residual Fill: Export:',
          'Albany Fill: Jacobian:',
          'Albany Jacobian Fill: Evaluate:',
          'Albany Jacobian Fill: Export:',
          'NOX Total Preconditioner Construction:',
          'NOX Total Linear Solve:')

# Extract file names and collect data
files = glob.glob('json/ctest-*')
df = json2dataframe(files, cases, nproc, names, timers, metadata)

# # Filter data by date if desired
# import datetime as dt
# df = df[df['date'] < dt.datetime.strptime('20191231', '%Y%m%d')]

In [13]:
# Set threshold for model.
threshold = 0.0001

# Number of days since most recent to warn about
recency = 10

In [14]:
print('Test cases:')
[print('  '+c) for c in cases]
print('Timers:')
[print('  '+n) for n in names]
print("Model threshold: %f" % threshold)

Test cases:
  ant-2-20km_ml_ls
  ant-2-20km_mu_ls
  ant-2-20km_mu_dls
  green-1-7km_fea_1ws
  green-1-7km_ml_ls_1ws
  green-1-7km_mu_ls_1ws
  green-1-7km_mu_dls_1ws
  green-1-7km_fea_mem
  green-1-7km_ml_ls_mem
  green-1-7km_mu_ls_mem
  green-1-7km_mu_dls_mem
Timers:
  Total Time
  Setup Time
  Total Fill Time
  Residual Fill
  Residual Fill Evaluate
  Residual Fill Export
  Jacobian Fill
  Jacobian Fill Evaluate
  Jacobian Fill Export
  NOX Total Preconditioner Construction
  NOX Total Linear Solve
Model threshold: 0.000100


## Performance Timelines

In [15]:
# Find changepoints and format data to work nicely with plots
seqs = {case:{} for case in cases}
mostRecent = df['date'].max()
events = {}
for case in cases:
    for name in names:
        # Detect changepoints
        data = df.loc[df['case']==case, ['date', name]].dropna()
        pts, _ = findChangePts(data[name].to_numpy(), threshold=threshold,
                               method='mean')
        
        # Look for recent changepoints to notify later
        for d in data['date'].iloc[pts]:
            if d in events.keys():
                events[d][case] = name
            else:
                events[d] = {case: name}
                    
        # Calculate mean/std between changepoints
        cols = ['date', name] + list(metadata)
        data = df.loc[df['case']==case, cols].dropna(subset=[name])
        data.reset_index(drop=True, inplace=True)
        data.rename(columns={name:'time'}, inplace=True)
        mean, std = regimeTimeseries(data['time'].to_numpy(), pts)
        temp = {'mean': mean, 'upper': mean+2*std, 'lower': mean-2*std}
        seqs[case][name] = pd.concat((data, pd.DataFrame(temp)), axis=1)
        
''''''
# Sort and print results
events = {k:events[k] for k in sorted(events.keys())}
print('Events in the most recent %d days:' % recency)
recentEvents = printEvents(events, mostRecent, recency)

Events in the most recent 10 days:
02/02/2020:
    green-1-7km_mu_ls_1ws: Residual Fill
    green-1-7km_mu_dls_1ws: Residual Fill
    green-1-7km_fea_mem: Residual Fill
    green-1-7km_mu_ls_mem: Residual Fill
    green-1-7km_mu_dls_mem: Residual Fill
02/08/2020:
    green-1-7km_mu_ls_1ws: Residual Fill Export
    green-1-7km_mu_dls_1ws: Residual Fill Export
02/09/2020:
    green-1-7km_ml_ls_1ws: Jacobian Fill Export
    green-1-7km_mu_dls_1ws: NOX Total Linear Solve
    green-1-7km_mu_dls_mem: NOX Total Linear Solve


In [16]:
lines = ['time', 'mean', 'upper', 'lower']
colors = ['blue', 'red', 'red', 'red']
modes = ['markers', 'lines', 'lines', 'lines']
dashes = ['solid', 'solid', 'dot', 'dot']

fig = go.Figure()
# Create series on plot
for line, color, mode, dash in zip(lines, colors, modes, dashes):
    for c in cases:
        if line == 'time':
            fig.add_trace(go.Scatter(
                x=seqs[c][names[0]]['date'],
                y=seqs[c][names[0]][line],
                mode=mode,
                line = dict(color=color, dash=dash, width=1.5),
                name=line,
                visible=True if c==cases[0] else False,
                customdata=seqs[c][names[0]].loc[:, ['date']+list(metadata)],
                hovertemplate=
                "Date: %{customdata[0]}<br>" +
#                 "Albany compiler: %{customdata[1]}<br>" +
                "Albany commit: %{customdata[2]}<br>" +
                "Trilinos commit: %{customdata[3]}" +
                "<extra></extra>",
            ))
        else:
            fig.add_trace(go.Scatter(
                x=seqs[c][names[0]]['date'],
                y=seqs[c][names[0]][line],
                mode=mode,
                line = dict(color=color, dash=dash, width=1.5),
                name=line,
                visible=True if c==cases[0] else False,
                hoverinfo='skip'
            ))

changedCases = {n for v in recentEvents.values() for n in v.keys()}

# Test case dropdown
caseOptions = [dict(
        args=['visible', [True if x==c else False for x in np.tile(cases, len(lines))]],
        label= '*'+c if c in changedCases else c,
        method='restyle'
    ) for c in cases]
    
# Timer dropdown
nameOptions = [dict(
        args=[{'x': [seqs[c][n]['date'] for _ in lines for c in cases],
               'y': [seqs[c][n][line] for line in lines for c in cases],
               'customdata': [seqs[c][n].loc[:, ['date']+list(metadata)].to_numpy() 
                              if line == 'time' else None
                              for line in lines for c in cases]}],
        label=n,
        method='restyle'
    ) for n in names]

# Add dropdowns to plot
fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=list(caseOptions),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,    xanchor="left",
            y=1.15, yanchor="top"
        ),
        go.layout.Updatemenu(
            buttons=list(nameOptions),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.3, xanchor="left",
            y=1.15, yanchor="top"
        ),
    ],
    xaxis_title='Simulation Date',
    yaxis_title='Wall-clock Time (s)'
)

iplot(fig)

### Plot of wall-clock times for nightly runs
Changepoints are estimated using a generalized likelihood ratio method on each timer, and then merged over all timers for a given test case. 
* Blue markers: recorded wall-clock time
* Solid red line: average wall-clock time between changepoints
* Dotted red lines: average wall-clock time $\pm$ two standard deviations

#### Plot window controls

* Test case and timer can be selected from the drop-down menus (* denotes recent changes detected)
* Hovering over data points shows various metadata
* Clicking on the legend will show/hide individual plot elements
* Click and drag to zoom in; double click to reset zoom

Pollak, Moshe; Siegmund, D. Sequential Detection of a Change in a Normal Mean when the Initial Value is Unknown. Ann. Statist. 19 (1991), no. 1, 394--416. doi:10.1214/aos/1176347990. https://projecteuclid.org/euclid.aos/1176347990

Siegmund, D.; Venkatraman, E. S. Using the Generalized Likelihood Ratio Statistic for Sequential Detection of a Change-Point. Ann. Statist. 23 (1995), no. 1, 255--271. doi:10.1214/aos/1176324466. https://projecteuclid.org/euclid.aos/1176324466

Hawkins, D. M., & Zamba, K. D. (2005). Statistical Process Control for Shifts in Mean or Variance using a Change Point Formulation. Technometrics, 47, 164-173.

Hawkins DM, Qiu P, Kang CW. The changepoint model for statistical process control. Journal of Quality Technology. 2003 Oct 1;35(4):355-366.

In [ ]:
data['date']
#         if d in events.keys():
#             events[d][case] = name
#         else:
#             events[d] = {case: name}

In [ ]:
pts